# Imoport necessary modules

In [ ]:
import numpy as np
import sep

import astropy.io
from astropy.io import fits 

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Set image dimensions with rcParams
rcParams['figure.figsize'] = [10., 8.]

# Open fits file and extract data

In [ ]:
#fits image declaration and array conversion

hdu_list = fits.open('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')
data = hdu_list[0].data
data = data.byteswap().newbyteorder()

# Display image

In [ ]:
#image view
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

# Extract background from image data

In [ ]:
#measure spacially varying background on the image
bkg = sep.Background(data)

# Display global background level and rms

In [ ]:
#'global' mean noise of image background
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate background as 2-d array maintaining size of original image
bkg_image = bkg.back()

# Display image of separated background

In [ ]:
#view the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
#evaluate background noise as 2-d array maintaining size of original image
bkg_rms = bkg.rms()

# Display image of background rms

In [ ]:
#view the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

# Separate background from image data

In [ ]:
#subtraction of background noise
data_sub = data - bkg

# Extract the objects from the image

In [ ]:
# Define source of objects, set threshold of object detection, set variance for detection threshold
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

# List the number of detected objects from the array

In [ ]:
# Display amount of objects detected as the length of the array
len(objects)

# Show the background-subtracted image and create shapes to highlight each detected object

In [ ]:
from matplotlib.patches import Ellipse

#plot background subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy = (objects['x'][i], objects['y'][i]),
               width = 6*objects['a'][i],
               height = 6*objects['b'][i],
               angle = objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

# Display the parameters extracted from the objects array

In [ ]:
#available fields
objects.dtype.names

# Extract the location and flux data from each object

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

# Create loop to display the flux from the first 10 detected objects

In [ ]:
#shows the first ten object results
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

# Convert flux data to AB magnitude, and create histogram to plot data

In [ ]:
data = hdu_list[0]
PHOTPLAM = data.header['PHOTPLAM']
PHOTFLAM = data.header['PHOTFLAM']
PHOTZPT = data.header['PHOTZPT']

#convert flux to ABMAG
abmag_zpt = -2.5 * np.log10(PHOTFLAM) - 21.10 - 5 * np.log10(PHOTPLAM) + 18.692
abmag = -2.5 * np.log10(np.fabs(flux)) + abmag_zpt

#create histogram of ABMAG flux conversions
plt.hist(abmag, range=[15,40], bins=300)
plt.figure(figsize=(0.05,0.05))
plt.show()